In [1]:
%load_ext autoreload
%autoreload 2

In [369]:
from bagsolver.bag import Bag
from bagsolver.base import BagBase
from bagsolver.genetic import GeneticBagSolver

from bagsolver.utils import load_bag_data, parse_solution

In [370]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [400]:
data = load_bag_data("data/NK/NK40_inst.dat", "data/NK/NK40_sol.dat")

In [431]:
defi, target = data[4]

In [432]:
target = int(target.split(" ")[2])

In [439]:
from tqdm import tqdm

from bagsolver.genetic import GeneticBagSolver

records = []

bag = GeneticBagSolver.from_line(defi, batch_size=30, mutation_rate=0.04)
pool = bag.new_pool_random()

for i in tqdm(range(1000)):
    batch_stats = bag.pool_fitness(pool)
    records.append((*batch_stats, bag.best_score))
    # print((*batch_stats, bag.best_score))
    pool = bag.genetic_iteration(pool)
    
    
print(target, bag.best_score)

df = pd.DataFrame(records, columns = ["pmin", "pmax", "pmean", "totmax"])

fig, ax = plt.subplots()
ax.hlines(y=target, xmin=0, xmax=len(df), colors='r', linestyles='--', lw=2)
df.plot(ax=ax)
plt.show()

  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: low >= high